In [ ]:
from azureml.core import Workspace, Dataset, Datastore

ws = Workspace(subscription_id='35d4664b-ed82-4074-b51a-a96056a154ac',
               resource_group='grupopruebatecnica',
               workspace_name='WS-PruebaTecnica')

datastore = Datastore.get(ws, "workspaceblobstore")

dataset = Dataset.File.from_files(path=(datastore, "workspaceblobstore/sentencias_pasadas.xlsx"))

local_path = dataset.download('.', overwrite=True)
local_path

In [ ]:
import zipfile
import xml.etree.ElementTree as ET
import csv

xlsx_path = "sentencias_pasadas.xlsx"
csv_path = "sentencias_pasadas.csv"

# --- Abrir XLSX como ZIP ---
with zipfile.ZipFile(xlsx_path) as z:
    sheet_xml = z.read('xl/worksheets/sheet1.xml')
    shared_xml = z.read('xl/sharedStrings.xml')

# --- Parsear sharedStrings.xml ---
shared_root = ET.fromstring(shared_xml)
ns = {'main': 'http://schemas.openxmlformats.org/spreadsheetml/2006/main'}

shared_strings = []
for si in shared_root.findall('main:si', ns):
    text_elements = si.findall('.//main:t', ns)
    text = ''.join([t.text for t in text_elements if t.text])  # unir si hay varios fragmentos
    shared_strings.append(text)

# --- Parsear sheet1.xml ---
sheet_root = ET.fromstring(sheet_xml)

rows = []
for row in sheet_root.findall('.//main:row', ns):
    values = []
    for cell in row.findall('main:c', ns):
        cell_type = cell.get('t')  # si t="s", es shared string
        value = cell.find('main:v', ns)

        if value is None:
            values.append("")
        else:
            if cell_type == "s":
                # índice a sharedStrings.xml
                idx = int(value.text)
                values.append(shared_strings[idx])
            else:
                # valor directo
                values.append(value.text)
    rows.append(values)

# --- Guardar CSV ---
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

print("CSV generado correctamente:", csv_path)

In [ ]:
import pandas as pd

df = pd.read_csv("sentencias_pasadas.csv", header=None)
df.head()

### **Modelo ChromaDB para realizar busquedas dentro del df usando Generative AI**

In [ ]:
%pip install sentence-transformers chromadb

### **Importar el Csv**

In [7]:
import pandas as pd

csv_path = "sentencias_pasadas.csv"
df = pd.read_csv(csv_path, header=None)

# Encabezados reales
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

# Detectar columna de fecha
fecha_col = [c for c in df.columns if "Fecha" in c][0]
df = df.rename(columns={fecha_col: "FechaRaw"})

# Renombrar otras columnas
df = df.rename(columns={
    "#": "idx",
    "Relevancia": "Relevancia",
    "Providencia": "Providencia",
    "Tipo": "Tipo",
    "Tema - subtema": "Tema",
    "resuelve": "Resuelve",
    "sintesis": "Sintesis"
})

# Limpiar textos
for c in ["Providencia", "Tipo", "Tema", "Resuelve", "Sintesis"]:
    df[c] = df[c].astype(str).str.strip()

# --------------------------
# CONVERSIÓN DE FECHA (DE TEXTO A DATETIME)
# --------------------------
df["FechaSentencia"] = pd.to_datetime(
    df["FechaRaw"],  # toma el valor original
    dayfirst=True,   # si las fechas están en dd/mm/yyyy
    errors="coerce"
)
df["FechaSentencia"] = df["FechaSentencia"].dt.strftime("%d/%m/%Y")

# Crear texto unificado
def make_doc(row):
    parts = [
        f"Providencia: {row.get('Providencia', '')}",
        f"Tipo: {row.get('Tipo', '')}",
        f"Fecha: {row.get('FechaSentencia', '')}",
        f"Tema: {row.get('Tema', '')}",
        f"Resumen: {row.get('Sintesis', '')}"
    ]
    return "\n".join(parts)

df["documentText"] = df.apply(make_doc, axis=1)

# Reordenar columnas
column_order = ["Relevancia", "Providencia", "Tipo", "FechaSentencia", "Tema", "Resuelve", "Sintesis"]
df = df[column_order + ["documentText"]]

print("Filas:", len(df))
df.head(3)

Filas: 329


,Relevancia,Providencia,Tipo,FechaSentencia,Tema,Resuelve,Sintesis,documentText
0,966965,T-185/22,nan,31/05/2022,nan,en nombre del pueblo y por mandato de la Const...,En este caso se formula la acción de tutela en...,Providencia: T-185/22\nTipo: nan\nFecha: 31/05...
1,963168,T-356/21,nan,15/10/2021,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...,Providencia: T-356/21\nTipo: nan\nFecha: 15/10...
2,956201,T-351/22,nan,07/10/2022,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...,Providencia: T-351/22\nTipo: nan\nFecha: 07/10...


In [12]:
# Sobrescribir el CSV original
df.to_csv("sentencias_pasadas.csv", index=False)

### **Chroma DB**

In [9]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# Modelo para generar embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Crear cliente Chroma
client = chromadb.Client()

# Crear colección
collection = client.create_collection(name="sentencias_legal")

# Insertar datos en la colección
for idx, row in df.iterrows():
    collection.add(
        ids=[str(idx)],
        documents=[row["documentText"]],
        metadatas=[{
            "Relevancia": row["Relevancia"],
            "Providencia": row["Providencia"],
            "Tipo": row["Tipo"],
            "FechaSentencia": row["FechaSentencia"],
            "Tema": row["Tema"],
            "Resuelve": row["Resuelve"],
            "Sintesis": row["Sintesis"]
        }],
        embeddings=[model.encode(row["documentText"]).tolist()]
    )


InternalError: Collection [sentencias_legal] already exists

In [10]:
# Crear o recuperar la colección existente
collection = client.create_collection(name="sentencias_legal", get_or_create=True)

In [11]:
for idx, row in df.iterrows():
    collection.add(
        ids=[str(idx)],
        documents=[row["documentText"]],
        metadatas=[{
            "Relevancia": row["Relevancia"],
            "Providencia": row["Providencia"],
            "Tipo": row["Tipo"],
            "FechaSentencia": row["FechaSentencia"],
            "Tema": row["Tema"],
            "Resuelve": row["Resuelve"],
            "Sintesis": row["Sintesis"]
        }],
        embeddings=[model.encode(row["documentText"]).tolist()]
    )

In [12]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Modelo para generar embeddings 
# model = SentenceTransformer('all-MiniLM-L6-v2')

# Función para buscar los casos más similares a un query
def buscar_casos(query, top_k=3):
    query_embedding = model.encode(query).tolist()
    
    # Buscar en Chroma
    resultados = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    # Extraer documentos y metadatos
    docs = resultados['documents'][0]
    metadatos = resultados['metadatas'][0]
    
    respuestas = []
    for doc, meta in zip(docs, metadatos):
        texto = (
            f"Providencia: {meta['Providencia']}\n"
            f"Tipo: {meta['Tipo']}\n"
            f"Fecha: {meta['FechaSentencia']}\n"
            f"Tema: {meta['Tema']}\n"
            f"Resumen: {meta['Sintesis']}"
        )
        respuestas.append(texto)
    return respuestas

In [13]:
# Tres demandas sobre redes sociales
print("=== Tres demandas sobre redes sociales ===")
casos_redes = buscar_casos("demandas relacionadas con redes sociales", top_k=3)
for i, c in enumerate(casos_redes, 1):
    print(f"\nCaso {i}:\n{c}")

# Caso de acoso escolar
print("\n=== Caso de acoso escolar ===")
casos_acoso = buscar_casos("acoso escolar", top_k=1)
for c in casos_acoso:
    print(c)

# Casos relacionados con PIAR
print("\n=== Casos relacionados con PIAR ===")
casos_piar = buscar_casos("PIAR", top_k=3)
for i, c in enumerate(casos_piar, 1):
    print(f"\nCaso {i}:\n{c}")

=== Tres demandas sobre redes sociales ===

Caso 1:
Providencia: T-087/23
Tipo: nan
Fecha: 28/03/2023
Tema: nan
Resumen: Las accionantes son periodistas que consideran haber sido víctimas de diferentes ataques en línea a través de la red social Twitter, de naturaleza misógina y de contenido sexualizado, que buscan infantilizar su oficio y censurarlas. En sede de tutela cuestionaron a la Dirección Nacional Electoral por no adoptar ninguna medida para hacer cesar la violencia, sancionar a los responsables y prevenirla. Así mismo, criticaron el hecho de que los partidos políticos y/o movimientos ciudadanos se hubieran favorecido de las agresiones, al alentarlas o tolerarlas. Se analiza temática relacionada con: 1°. La violencia en línea contra mujeres periodistas. 2°. El derecho a la libertad de expresión en las redes sociales y, 3°. El régimen sancionatorio de los actores, partidos y movimientos políticos. A pesar de no encontrar demostrada la vulneración de los derechos fundamentales de